# Environement Setup

In [6]:
import sys, os
try:
    from google.colab import drive, userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

REPO_NAME = 'MistakeDetection'

if IS_COLAB:
    print("☁️ Colab rilevato.")
    if not os.path.exists('/content/drive'): drive.mount('/content/drive')

    GITHUB_USER = 'MarcoPernoVDP'
    try:
        TOKEN = userdata.get('GITHUB_TOKEN')
        REPO_URL = f'https://{TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git'
    except:
        REPO_URL = f'https://github.com/{GITHUB_USER}/{REPO_NAME}.git'

    ROOT_DIR = f'/content/{REPO_NAME}'
    if not os.path.exists(ROOT_DIR):
        !git clone {REPO_URL}
    else:
        %cd {ROOT_DIR}
        !git pull
        %cd /content

    
else:
    print("Ambiente locale rilevato.")
    ROOT_DIR = os.getcwd()
    while not os.path.exists(os.path.join(ROOT_DIR, '.gitignore')) and ROOT_DIR != os.path.dirname(ROOT_DIR):
        ROOT_DIR = os.path.dirname(ROOT_DIR)

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)


Ambiente locale rilevato.


# Dataset Setup

In [7]:
from utils import setup_project

# Esegue: Setup Dati (unzip/copy), Login WandB, Setup Device
device = setup_project.initialize(ROOT_DIR)

# Ora puoi passare agli import del modello
from dataset.capitain_cook_4d_dataset import CaptainCook4D_Dataset
from models.BaselineV1_MLP import BaselineV1_MLP

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\dylan\_netrc


Setup Progetto in: c:\Users\dylan\Desktop\Università\MistakeDetection
source_path: c:\Users\dylan\Desktop\Università\MistakeDetection\_file
Setup Dati da: c:\Users\dylan\Desktop\Università\MistakeDetection\_file
Inizio setup dati...
   Sorgente: c:\Users\dylan\Desktop\Università\MistakeDetection\_file
   Destinazione: c:\Users\dylan\Desktop\Università\MistakeDetection\data
Copia cartella: annotation_json...
Copia cartella: omnivore...
✅ Setup completato! Dati pronti in: c:\Users\dylan\Desktop\Università\MistakeDetection\data
WandB Logged in.
Device: cuda


# Dataset Split

In [8]:
import os
from dataset.capitain_cook_4d_dataset import CaptainCook4D_Dataset, DatasetSource
from dataset.utils import get_loaders

try:
    full_dataset = CaptainCook4D_Dataset(dataset_source=DatasetSource.OMNIVORE, root_dir=ROOT_DIR)
    train_loader, val_loader, test_loader = get_loaders(
        full_dataset,
        batch_size=512,
        seed=42
    )

except Exception as e:
    print(f"❌ Errore: {e}")

Loading from: c:\Users\dylan\Desktop\Università\MistakeDetection\data\omnivore...

DATASET INFO
   Shape: torch.Size([9798, 1024]) -> 9798 Campioni, 1024 Features
FULL DATASET       | Tot: 9798   | OK: 5970  (60.9%) | ERR: 3828  (39.1%) | Ratio: 1:1.6
-----------------------------------------------------------------
TRAIN SET          | Tot: 6860   | OK: 4173  (60.8%) | ERR: 2687  (39.2%) | Ratio: 1:1.6
VALIDATION SET     | Tot: 979    | OK: 603   (61.6%) | ERR: 376   (38.4%) | Ratio: 1:1.6
TEST SET           | Tot: 1959   | OK: 1194  (60.9%) | ERR: 765   (39.1%) | Ratio: 1:1.6



In [10]:
from utils.inspect_npz import inspect_npz_from_dataset

dataset_folder = "omnivore"
npz_filename = "1_7_360p.mp4_1s_1s.npz"

# Ispezione del file .npz
inspect_npz_from_dataset(ROOT_DIR, dataset_folder, npz_filename, n_rows=3)



File: c:\Users\dylan\Desktop\Università\MistakeDetection\data\omnivore\1_7_360p.mp4_1s_1s.npz
Chiavi presenti nel file: ['arr_0']

Array 'arr_0' - shape: (604, 1024), dtype: float32
[[ 0.6910985   0.09298898 -0.6608225  ... -0.75679165  1.2401273
  -0.5683658 ]
 [ 0.40254688 -0.4466254  -0.8645446  ... -1.2709565   0.7917245
  -0.5052321 ]
 [ 0.643613   -0.48683766 -0.88651866 ... -1.0358062   0.658605
  -0.27201462]]


# MLP (Version 1)

In [ ]:
import torch
import torch.nn as nn

model = BaselineV1_MLP(1024).to(device)

In [ ]:
lr = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr)

# Quanto pesa la classe “positiva” = classe "1" = classe "error":
# - CASO 1: rapporto effettivo del dataset
#train_pos_weight = train_cnt_0 / train_cnt_1

# - CASO 2: rapporto usato nel paper
train_pos_weight = 1.5

print(f"Peso classe positiva: {train_pos_weight}")
train_pos_weight = torch.tensor([train_pos_weight], device=device)

criterion = nn.BCEWithLogitsLoss(pos_weight=train_pos_weight)

epochs = 50

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

for epoch in range(epochs):

    # -------------------------
    #        TRAIN
    # -------------------------
    model.train()
    total_loss = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device).float()

        outputs = model(inputs)            # [B, 1]
        outputs = outputs.squeeze(1)       # [B]

        loss = criterion(outputs, labels)
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_loader)

    # -------------------------
    #        EVAL
    # -------------------------
    model.eval()
    total_val_loss = 0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device).float()

            outputs = model(inputs).squeeze(1)  # logits

            # same loss as train
            val_loss = criterion(outputs, labels)
            total_val_loss += val_loss.item()

            # convert logits → probabilities → binary predictions
            probs = torch.sigmoid(outputs)
            preds = (probs >= 0.5).long()

            all_preds.append(preds.cpu())
            all_targets.append(labels.cpu())

    # concat
    all_preds = torch.cat(all_preds).numpy()
    all_targets = torch.cat(all_targets).numpy()

    avg_val_loss = total_val_loss / len(test_loader)
    acc = accuracy_score(all_targets, all_preds)
    f1  = f1_score(all_targets, all_preds, zero_division=0)

    print(f"Epoch {epoch+1}/{epochs} "
          f"- Train Loss: {avg_train_loss:.4f} "
          f"- Val Loss: {avg_val_loss:.4f} "
          f"- Acc: {acc:.4f} "
          f"- F1: {f1:.4f}")
